I'm running the code again for `create_arpas` but with new data! Creating binary models vs full grade levels

In [1]:
from article_process_new import ArticleLM

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
import pandas as pd

In [3]:
# path_to_data = '/Users/stephanie/data/newsela_articles_corpus_2019-03-25_SMALL'
path_to_data = '/Users/stephanie/data/newsela_articles_corpus_2019-03-25'
path_to_kenlm = '/Users/stephanie/github/kenlm'
path_to_arpa = path_to_kenlm + '/lm'

In [4]:
# pd.read_csv('/Users/stephanie/data/newsela_articles_corpus_2019-03-25/MANIFEST.csv')

In [6]:
articleLM = ArticleLM(path_to_data, path_to_kenlm, path_to_arpa, 5, 'binary')

In [ ]:
articleLM.metadata_split

## Builds the processed sentences that I need for KenLM

In [7]:
articleLM.build_data()

In [8]:
articleLM.train_all_arpas()

Processing easy
Grade easy has 470591 sentences in the train set.
=== 1/5 Counting and sorting n-grams ===
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
Unigram tokens 5549275 types 57394
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:688728 2:1340800512 3:2514001152 4:4022401792 5:5866002944
Statistics:
1 57394 D1=0.575252 D2=0.998303 D3+=1.60475
2 861613 D1=0.703198 D2=1.15485 D3+=1.49267
3 2340349 D1=0.805079 D2=1.27125 D3+=1.50709
4 3308432 D1=0.873327 D2=1.41866 D3+=1.56887
5 3582139 D1=0.7309 D2=1.67816 D3+=2.07964
Memory estimate for binary LM:
type     MB
probing 211 assuming -p 1.5
probing 249 assuming -r models -p 1.5
trie     99 without quantization
trie     53 assuming -q 8 -b 8 quantization 
trie     88 assuming -a 22 array pointer compression
trie     42 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:688728 2:13785808 3:468

## Printing out stats

how many sentences there are in training set, indexed by grade level?

In [9]:
for g in articleLM.level_sentences.get('train').keys():
    print(g, len(articleLM.level_sentences.get('train').get(g)))

easy 470591
hard 349293


In [10]:
# Number of sentences, train
sentence_len = 0
for g in articleLM.level_sentences.get('train').keys():
    sentence_len+=len(articleLM.level_sentences.get('train').get(g))
    
print(sentence_len)

819884


In [11]:
# Number of sentences, validation
sentence_len = 0
for g in articleLM.level_sentences.get('val').keys():
    sentence_len+=len(articleLM.level_sentences.get('val').get(g))
print(sentence_len)

102903


how many articles there are in each set, indexed by grade level?

In [12]:
articleLM.metadata_split[articleLM.metadata_split.train_val_test=='train'].grade_level.value_counts()

5    3910
3    3286
7    2883
4    2560
9    2028
6    1839
8    1592
2    1029
Name: grade_level, dtype: int64

In [13]:
articleLM.metadata_split[articleLM.metadata_split.train_val_test=='train'].grade_level.value_counts().sum()

19127

In [14]:
articleLM.metadata_split[articleLM.metadata_split.train_val_test=='val'].grade_level.value_counts()

5    507
3    427
7    346
4    300
6    244
9    235
8    188
2    144
Name: grade_level, dtype: int64

## Comparing perplexities - Sentence Level

In [15]:
perplex_guesses = articleLM.compute_all_sentences_best_guess(['easy', 'hard'])

In [16]:
len(perplex_guesses[perplex_guesses.level == perplex_guesses.best_guess] ) * 1. / len(perplex_guesses)

0.6743340816108374

In [17]:
perplex_guesses.head()

,level,best_guess
0,easy,easy
1,easy,easy
2,easy,easy
3,easy,easy
4,easy,hard


## Article level

In [14]:
def compute_article_best_guess():
    article_sentences = articleLM.article_val_sentences

    iter = 0
    article_df = pd.DataFrame({})
    for article_gl in article_sentences.keys():
        #     mean_guess, median_guess = articleLM.compute_perplexity_entire_article(article_sentences[article_gl])

        sample_perp = dict()
        for ix, sentence in enumerate(article_sentences[article_gl]):
            perplexities = articleLM.compute_sentence_perplexities(sentence)
            sample_perp[ix] = perplexities
        sample_perp_df = pd.DataFrame(sample_perp).T
        sample_perp_df.columns = sorted(articleLM.models.keys())

        sample_perp_df.loc[:, 'min_perplexity'] = sample_perp_df.idxmin(axis=1, skipna=True)
        sample_perp_df.loc[:, 'true_gl'] = article_gl[1]
        sample_perp_df.loc[:, 'article'] = article_gl[0]

        article_df = pd.concat([article_df, sample_perp_df])

        iter += 1

        if iter % 10 == 0:
            print("iteration {}, article {}".format(iter, article_gl))

    return article_df

# def plot_best_article_guess():
#     article_df = articleLM.compute_article_best_guess()
#     means = article_df.groupby(['article', 'true_gl']).mean()['min_perplexity'].reset_index()
#     means.rename(columns={'min_perplexity': 'predicted_gl'}, inplace=True)

#     fig = plt.figure(figsize=(10, 8))
#     ax = fig.gca()
#     ax.set_title("Grade Level Distributions")
#     # ax.set_xlabel("True Grade Level")
#     # ax.set_ylabel("Predicted Grade Level")
#     sns.boxplot(ax=ax, x="true_gl", y="predicted_gl", data=means)
#     ax.set_xlabel("True Grade Level")
#     ax.set_ylabel("Predicted Grade Level, using Means")

#     return article_df

In [19]:
most_common = articleLM.compute_article_best_guess_binary()

iteration 10, article ('gerrymandering-redistricting', 'easy')
iteration 20, article ('hawaii-philosophical', 'easy')
iteration 30, article ('nobel-literature', 'easy')
iteration 40, article ('mexico-earthquake', 'easy')
iteration 50, article ('parents-let-teens-ride-uber', 'easy')
iteration 60, article ('mummy-scans', 'easy')
iteration 70, article ('tortoise-shells', 'hard')
iteration 80, article ('teen-nasajob', 'easy')
iteration 90, article ('tsunami-pod', 'easy')
iteration 100, article ('netflix-fortnite', 'easy')
iteration 110, article ('preslibrary-land', 'easy')
iteration 120, article ('nutria-laststand', 'hard')
iteration 130, article ('pokemon-augmented-reality', 'hard')
iteration 140, article ('opinion-how-transform-education-system', 'easy')
iteration 150, article ('teens-sleep', 'easy')
iteration 160, article ('toyota-flying-car-and-luxury-boat', 'hard')
iteration 170, article ('kids-hugging-holidays', 'easy')
iteration 180, article ('mlb-steroids', 'easy')
iteration 190, a

iteration 1480, article ('crow-funerals', 'easy')
iteration 1490, article ('video-games-kids-brains', 'hard')
iteration 1500, article ('not-all-teenagers-drink', 'hard')
iteration 1510, article ('end-of-world-records', 'easy')
iteration 1520, article ('elem-ocean-hot-spots', 'easy')
iteration 1530, article ('mars-ice-cap-water', 'easy')
iteration 1540, article ('deep-sea-creature-discovery', 'hard')
iteration 1550, article ('elem-part-time-hermit', 'easy')
iteration 1560, article ('beasts-death-valley', 'hard')
iteration 1570, article ('elem-dog-true-cost', 'easy')
iteration 1580, article ('unique-giant-sloth-fossil', 'hard')
iteration 1590, article ('religion-accommodation', 'hard')
iteration 1600, article ('elem-manta-ray-nursery-discovery', 'easy')
iteration 1610, article ('elem-hero-dogs-program', 'easy')
iteration 1620, article ('fat-bear-week', 'easy')
iteration 1630, article ('new-women-congress-fashion-statement', 'hard')
iteration 1640, article ('fall-equinox', 'easy')
iterati

## Confusion matrix

In [22]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

cm = confusion_matrix(most_common.true_gl, most_common.predicted_gl)

In [23]:
cm

array([[702, 338],
       [126, 661]])

In [24]:
accuracy_score(most_common.true_gl, most_common.predicted_gl)

0.746031746031746

In [25]:
f1_score(most_common.true_gl, most_common.predicted_gl, pos_label='easy')

0.7516059957173448